## 1. Librería

*   Frecuencia aproximada de un ítem:
    La librería permite consultar cuántas veces aparece un ítem en el flujo de datos, sin necesitar almacenar cada aparición exactamente. Para esta tarea se utilizará la estructura de Count-Min Sketch, que estima la frecuencia con un margen de error controlado y poco uso de memoria.

*   Elementos más frecuentes (Top-k):
    La librería permite identificar los ítems más repetidos dentro del flujo, para conocer las tendencias. Para esta tarea se utilizará la estructura de Space-Saving, que mantiene un resumen compacto de los ítems más frecuentes y sus estimaciones.

*   Estimación de momentos:
    La librería permite calcular métricas estadísticas como la varianza de las frecuencias, para analizar la dispersión o desigualdad en los datos. Para esta tarea se utilizará la estructura de AMS Sketch (Alon-Matias-Szegedy), que ofrece estimaciones eficientes y con alta precisión.
    
*   Número de elementos únicos:
    La librería permite estimar la cantidad de ítems distintos que han aparecido en el flujo, sin necesidad de almacenarlos todos. Para esta tarea se utilizará la estructura de HyperLogLog++, que ofrece una estimación precisa de la cardinalidad con muy poco uso de memoria.

*   Agregaciones sobre ventanas deslizantes:
    La librería permite hacer consultas sobre los últimos n ítems del flujo, como la suma o promedio en la ventana más reciente. Para esta tarea se utilizará la estructura de Sliding Window Aggregation, que actualiza estos valores de forma dinámica.



## 2. Estructuras Implementadas

### 2.1. Count-Min Sketch
Permite estimar la frecuencia de cualquier clave en el flujo de datos usando múltiples funciones de hash.

Cada vez que se recibe una clave, se calcula un índice para cada fila del sketch y se incrementa el contador correspondiente.  
Esto garantiza que, aunque algunos contadores puedan sobrestimar, la estimación final se aproxima bien gracias a usar el mínimo entre las filas.

Complejidad:
* La inserción y la estimación dependen del número de funciones de hash:  $
\boxed{O(\text{número de hashes})}
$

### 2.2. Space-Saving (Stream Summary)
Es una estructura diseñada para encontrar los elementos más frecuentes (top-k) en un flujo de datos con memoria limitad

* Cuando agregamos un nuevo elemento
Si ya está en el resumen: incrementa su contador.

* Si no está y hay espacio: lo añade con contador 1.

* Si no está y no hay espacio: reemplaza el elemento con menor frecuencia, sumando ese contador + 1 para capturar el posible error.

De esta forma, siempre se priorizan los elementos más frecuentes en el resumen.+

Complejidad

* Inserción: 𝑂
(
𝑘
)

### 2.3. AMS Sketch
Estima el segundo momento o varianza de frecuencias de un flujo de datos.
#### Función signo
Para cada clave ingresada se obtiene un signo
$g(x) =
\begin{cases}
+1 & \text{con probabilidad } 0.5 \\
-1 & \text{con probabilidad } 0.5
\end{cases}$
Luego, se actuliza un contador
$Z \leftarrow Z + g(x)$

#### Estimador con un contador

Luego de procesar el flujo, se calcula:

$\hat{F}_2 = Z^2$

Este valor es un estimador insesgado de $ F_2 $, , pero puede tener varianza alta.

#### Estimador con varios contadores
Para reducir la varianza, se utilizan varios contadores independientes $(Z_1, Z_2, \ldots, Z_r )$, cada uno con su propia función de signo:

$
\hat{F}_2 = \frac{1}{r} \sum_{i=1}^{r} Z_i^2
$

Esto asegura un estimador más estable y con menor error relativo.

##### Complejidad  
* Tanto para la inserción como para la estimación, la complejidad es proporcional al número de contadores que usemos.  
$
\boxed{O(\text{n})}
$



### 2.4. HyperLogLog++
Cada vez que se recibe un nuevo elemento, se calcula un hash uniforme de 64 bits.  
Se usan los primeros bits para decidir qué registro actualizar y el resto para determinar cuántos ceros consecutivos hay.  
Se actualiza el registro correspondiente con la mayor cantidad de ceros observada.
Combina todos los registros para dar una aproximación muy precisa del número de elementos distintos en el flujo, usando técnicas de corrección.

* Complejidad:
Tanto la actualización como la estimación final dependen del número de registros (buckets):  
$
\boxed{O(\text{número de buckets})}
$

### 2.5. Sliding Window Aggregation
Permite realizar consultas agregadas (por ejemplo, suma, promedio) en una ventana deslizante de tamaño fijo.


Usa una estructura como una deque para almacenar solo los últimos N elementos.


* Mantiene los últimos N elementos en un deque (doble cola eficiente para inserciones y eliminaciones en ambos extremos).
* A medida que llegan nuevos elementos, añade el nuevo elemento al final de la ventana y elimina el m+as antiguo.
* Calcula la suma acumulada y, a partir de esta, el promedio de los últimos elementos.
* Permite consultar métricas básicas de la ventana: suma, promedio, tamaño actual, máximo y mínimo.




Complejidad
* Inserción y eliminación (por deque): 𝑂
(
1
)

* Cálculo de promedio (usando suma acumulada): 𝑂
(
1
)

* Consulta de máximo y mínimo:𝑂
(
𝑁
)




